## Introduction
First we need to import the necessary libraries to use for this lab. We'll be using [`numpy`](http://www.numpy.org/) for linear algebra operations, [`matplotlib`](https://matplotlib.org/) for plotting, [`scipy`](https://docs.scipy.org/doc/scipy/reference/) for scientific and numerical computations, and [`tensorflow`](https://www.tensorflow.org/) for computing the neural networks. We collaborated using our [github repository](https://github.com/CoeWorl/ML-Lab-04) where all the code and datasets are located, and instructions to install necessary libraries at Andrew Ng's [github repository](https://github.com/dibgerge/ml-coursera-python-assignments) and the instructions to install the [tensorflow](https://www.tensorflow.org/install) library.

In [34]:
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np

# Plotting library
from matplotlib import pyplot

# Optimization module in scipy
from scipy import optimize

# Computing neural network layers
import tensorflow as tf

# Parsing the .tsv/.csv files
import pandas as pd

#### Importing the data
Using pandas to create the dataframe and creating the arrays for the data.

In [35]:
df = pd.read_csv('localify_music_genre - song_dataset_short_limited.csv')

#Testing to see what the dataframe looks like
#print(df.head()) #to test the dataframe

X, y = df.iloc[:, 2:10], df.iloc[:, 12]


In [36]:
df.head()

,song_name,artist_name,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,genre_name,Unnamed: 12
0,All Mixed Up,311,0.010400,0.760,0.702,0.00000,0.380,-9.404,0.0716,92.323998,0.659,alternative,0
1,Don't Tread On Me,311,0.000045,0.574,0.919,0.00280,0.116,-6.418,0.0737,176.531998,0.704,alternative,0
2,Beautiful Disaster,311,0.000387,0.576,0.738,0.00131,0.130,-8.122,0.0353,168.132004,0.675,alternative,0
3,Backfired,408,0.076700,0.496,0.937,0.00000,0.123,-2.628,0.1320,178.020996,0.182,punk,8
4,Manic,408,0.001370,0.436,0.980,0.00000,0.638,-3.491,0.2220,200.097000,0.674,punk,8


In [37]:

import random
#create df for training
trainData = df.copy()

#calculate length of validation df
lenValidation = len(df)*0.2

#create empty df for validation
validationData = df.iloc[0:0]

#run until validationData is 20% of original df
while len(validationData) != int(lenValidation):
    #find random index from df
    randomIndex = random.randint(0,len(trainData)-1)
    #get row at index
    row = trainData.iloc[randomIndex]
    #extract artist name
    artist_name = row['artist_name']
    #extract all indices with that artist name
    artist_rows = trainData[trainData['artist_name'] == artist_name]
    #if there is enough space in validationData for them all add to validation
    if(lenValidation-len(validationData) >= len(artist_rows)):
        #add rows to validation df
        validationData = pd.concat([validationData, artist_rows], ignore_index=True)
        #remove validation rows from traindata
        trainData = trainData[trainData['artist_name'] != artist_name]

print(len(trainData))
print(len(validationData))

8000
2000
